# Conversational Memory for LangChain

Conversational memory allows our chatbots and agents to remember previous interactions within a conversation. Without conversational memory, our chatbots would only ever be able to respond to the last message they received, essentially forgetting all previous messages with each new message.

Naturally, conversations require our chatbots to be able to respond over multiple interactions and refer to previous messages to understand the context of the conversation.

> ⚠️ **Important**

Although it is currently recommended to use the **LangGraph** library to implement chats with memory — since it allows you to efficiently work with complex workflows (for example: 🗓️ saving notes to Google Calendar, 🗄️ querying relational databases, and 🤖 using MCPs) — in this notebook we will use **LangChain** to explain the basic concepts of conversational memory.

This will make it easier to understand before moving on to more advanced tools.

## LangChain's Memory Types
LangChain versions `0.0.x` consisted of various conversational memory types. Most of these are due for deprecation but still hold value in understanding the different approaches that we can take to building conversational memory.

Throughout the notebook we will be referring to these older memory types and then rewriting them using the recommended RunnableWithMessageHistory class. We will learn about:

- **ConversationBufferMemory:** the simplest and most intuitive form of conversational memory, keeping track of a conversation without any additional bells and whistles.
- **ConversationBufferWindowMemory:** similar to ConversationBufferMemory, but only keeps track of the last k messages.
- **ConversationSummaryMemory:** rather than keeping track of the entire conversation, this memory type keeps track of a summary of the conversation.
- **ConversationSummaryBufferMemory:** merges the ConversationSummaryMemory and ConversationTokenBufferMemory types.

We'll work through each of these memory types in turn, and rewrite each one using the RunnableWithMessageHistory class.

In [28]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
os.environ["LANGSMITH_PROJECT"] = "llm-training-05-rag-p4"

llm =  ChatOpenAI(
    model="gpt-4.1-nano",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.7,
    verbose=True
)

### 1. ConversationBufferMemory with RunnableWithMessageHistory
When implementing `unnableWithMessageHistory` we will use LangChain Expression Language **(LCEL)** and for this we need to define our prompt template and LLM components.

In [46]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Zeta."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])
pipeline = prompt_template | llm


Our `RunnableWithMessageHistory` requires our pipeline to be wrapped in a `RunnableWithMessageHistory` object. This object requires a few input parameters. One of those is `get_session_history`, which requires a function that returns a `ChatMessageHistory` object based on a session ID. We define this function ourselves:

In [47]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

We also need to tell our runnable which variable name to use for the chat history (ie history) and which to use for the user's query (ie query).

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [53]:
# Test the pipeline with history
pipeline_with_history.invoke(
    {"query": "I work in BTS"},
    config={"session_id": "id_124"}
)

AIMessage(content="That's great! Working with BTS must be an exciting experience, whether you're in management, production, marketing, or another area. How can I assist you related to your work with BTS?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 24, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f12167b370', 'id': 'chatcmpl-C46WvG3JCUSGSanG3RamftuqFH5Ai', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--19c8ef84-3bd2-47c8-a409-edf1fc9c2997-0', usage_metadata={'input_tokens': 24, 'output_tokens': 37, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [55]:
chat_map["id_123"].messages

[HumanMessage(content='where is the library?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Could you please specify which library you're referring to or provide your current location? That way, I can help you find the nearest library.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 25, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C46WDUq4t5FiWLxNHG4Rd43UCPlcU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1ad1b5c0-223b-443f-b4b1-7035ccf4cf5e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 27, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_deta

In [59]:
# Test the pipeline with history, getting a response using the same session ID
pipeline_with_history.invoke( {"query": "What's my name?"}, config={"session_id": "id_123"})

AIMessage(content='Your name is Roger.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 281, 'total_tokens': 286, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C46Xvi4OTTtzafYfB46QRsLgFUBaR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d54d5301-8475-401d-a1a4-0e4639d61684-0', usage_metadata={'input_tokens': 281, 'output_tokens': 5, 'total_tokens': 286, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [58]:
# Test the pipeline with history, getting a response using a new session ID
pipeline_with_history.invoke({"query": "What's my name?"}, config={"session_id": "id_456"})

AIMessage(content='Your name is Roger. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 257, 'total_tokens': 269, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C46XkSRtVJ3iakSIk7v92b4UfiEVp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1c917f3-71fb-4a1f-9308-60a63ccd9050-0', usage_metadata={'input_tokens': 257, 'output_tokens': 12, 'total_tokens': 269, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

We will use **PostgreSQL** for this example, with a database hosted on **NeonDB**.

In [60]:
# Create the table (if it doesn't exist) to store chat history
from langchain_postgres import PostgresChatMessageHistory
import psycopg
import os

CONNECTION_STRING = os.getenv("POSTGRESQL_CONNECTION_STRING")
sync_connection = psycopg.connect(CONNECTION_STRING)

table_name = "chat_history"
PostgresChatMessageHistory.create_tables(sync_connection, table_name)

In [61]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
import uuid

# Replace with your PostgreSQL credentials
session_id = str(uuid.uuid4())

# Initialize the chat history manager
chat_history = PostgresChatMessageHistory(
    table_name,
    session_id,
    sync_connection=sync_connection
)

# To simulate a conversation, we can add some initial messages to the chat history
chat_history.add_messages(
    [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content="Hello My name is Roger I live in Cajamarca, Peru, how are you?"),
        AIMessage(content="I'm doing well, thank you! How can I assist you today?"),
        HumanMessage(content="What is my name?"),
        AIMessage(content="Your name is Roger."),
        HumanMessage(content="Where do I live?"),
        AIMessage(content="You live in Cajamarca, Peru."),
        HumanMessage(content="How old am I?"),
        AIMessage(content="I'm not sure how old you are, but I can help you with other questions."),
    ]
)


pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    lambda s: PostgresChatMessageHistory(table_name, s, sync_connection=sync_connection),
    input_messages_key="query",   # must match the prompt input variable
    history_messages_key="history"  # must match the prompt history variable
)

pipeline_with_history.invoke(
    {"query": "Hello, how are you?"},
    config={"configurable": {"session_id": session_id}}
)

AIMessage(content="Hello! I'm doing well, thank you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 144, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C46a6IkqaGAMCFWZe5KoqNzkcJMWL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--64f1e9a5-e925-4eae-bc2e-5f516f925a22-0', usage_metadata={'input_tokens': 144, 'output_tokens': 16, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [62]:
session_id

'f2841b21-cf50-43f8-8614-700d4aad4190'

We will use **DynamoDB** for this example

creating table in dynamodb if not exists
> ⚠️ **Important:**  
To create DynamoDB tables programmatically, your AWS user must have the necessary IAM permissions (e.g., `dynamodb:CreateTable`). Make sure your credentials allow table creation, otherwise you may encounter permission errors.

In [63]:
import boto3

# Get the service resource.
dynamodb = boto3.resource("dynamodb")
dynamo_table_name = "ChatBotSessionTable"
try:
    # Create the DynamoDB table.
    table = dynamodb.create_table(
        TableName=dynamo_table_name,
        KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )

    # Wait until the table exists.
    table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

    # Print out some data about the table.
    print(table.item_count)
except dynamodb.meta.client.exceptions.ResourceInUseException:
    # If the table already exists, we can just get it
    table = dynamodb.Table(dynamo_table_name)
    print("Table already exists, using existing table.")

Table already exists, using existing table.


In [64]:
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
import uuid

# Replace with your PostgreSQL credentials
session_id = str(uuid.uuid4())

chat_history = DynamoDBChatMessageHistory(table_name=dynamo_table_name, session_id=session_id)

chat_history.add_messages(
    [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content="Hello My name is Roger I live in Cajamarca, Peru, how are you?"),
        AIMessage(content="I'm doing well, thank you! How can I assist you today?"),
        HumanMessage(content="What is my name?"),
        AIMessage(content="Your name is Roger."),
        HumanMessage(content="Where do I live?"),
        AIMessage(content="You live in Cajamarca, Peru."),
        HumanMessage(content="How old am I?"),
        AIMessage(content="I'm not sure how old you are, but I can help you with other questions."),
    ]
)


pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    lambda s: chat_history,
    input_messages_key="query",   # must match the prompt input variable
    history_messages_key="history"  # must match the prompt history variable
)

pipeline_with_history.invoke(
    {"query": "Hello, how are you?"},
    config={"configurable": {"session_id": session_id}}
)

AIMessage(content="Hello! I'm doing well, thank you for asking. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 144, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C46ems115ggpOy26li3LXsx7yJALL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d25221ed-a91a-411a-bc28-9c55bab809d2-0', usage_metadata={'input_tokens': 144, 'output_tokens': 18, 'total_tokens': 162, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [65]:
pipeline_with_history.invoke(
    {"query": "Donde vivo?"},
    config={"configurable": {"session_id": session_id}}
)

AIMessage(content='Vives en Cajamarca, Perú.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 174, 'total_tokens': 182, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2f8f', 'id': 'chatcmpl-C46epMENoUxg1uPhrOt9GsWkG43Py', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--320a706e-f069-44a9-a982-6973b296d5a1-0', usage_metadata={'input_tokens': 174, 'output_tokens': 8, 'total_tokens': 182, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 2. ConversationBufferWindowMemory
The `ConversationBufferWindowMemory` type is similar to `ConversationBufferMemory`, but only keeps track of the **last k messages**. There are a few reasons why we would want to keep only the last k messages:

⚡️ **Very Important**: More messages mean more tokens are sent with each request, and more tokens increases latency and cost. 💸⏳

LLMs tend to perform worse when given more tokens, making them more likely to deviate from instructions, hallucinate, or "forget" information provided to them. Conciseness is key to high performing LLMs.

If we keep all messages we will eventually hit the LLM's context window limit, by adding a window size k we can ensure we never hit this limit.

The buffer window solves many problems that we encounter with the standard buffer memory, while still being a very simple and intuitive form of conversational memory.

In [70]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3, return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is Roger")
memory.chat_memory.add_ai_message("Hey Roger, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory stores the last k messages, dropping the rest.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Very cool!', additional_kwargs={}, response_metadata={})]}

In [69]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)
chain.invoke({"input": "what is my name again?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is Roger', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey Roger, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kw

{'input': 'what is my name again?',
 'history': [HumanMessage(content='Hi, my name is Roger', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Roger, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}

### 3. ConversationSummaryMemory
Next up we have ConversationSummaryMemory, this memory type keeps track of a summary of the conversation rather than the entire conversation. This is useful for long conversations where we don't need to keep track of the entire conversation, but we do want to keep some thread of the full conversation.

As before, we'll start with the original memory class before reimplementing it with the RunnableWithMessageHistory class.

In [71]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)
chain = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)


In [72]:
chain.invoke({"input": "hello there my name is James"})
chain.invoke({"input": "I am researching the different types of conversational memory."})
chain.invoke({"input": "I have been looking at ConversationBufferMemory and ConversationBufferWindowMemory."})
chain.invoke({"input": "Buffer memory just stores the entire conversation"})
chain.invoke({"input": "Buffer window memory stores the last k messages, dropping the rest."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello there my name is James
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human introduces themselves as James, and the AI greets him warmly, asking about his day and offering assistance or topics to discuss.
Human: I am researching the different types of conversational memory.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after forma

{'input': 'Buffer window memory stores the last k messages, dropping the rest.',
 'history': 'The human, James, mentions researching ConversationBufferMemory and ConversationBufferWindowMemory, and the AI explains that ConversationBufferMemory stores the entire conversation history to maintain full context, aiding coherence in complex interactions, but it can increase memory usage; the AI asks if James is exploring these for a specific project or general understanding.',
 'response': 'Exactly! ConversationBufferWindowMemory is designed to keep only the most recent k messages in memory, discarding older parts of the conversation as new messages arrive. This approach helps manage memory usage more efficiently, especially in long interactions, by limiting the amount of context the model needs to process at once. For example, if you set a window size of 5, then only the last 5 messages—be they user inputs or AI responses—are retained, while earlier parts are forgotten. This is particularly

# Summary Notes: Conversational Memory in LangChain

This notebook explored the concept of conversational memory in LangChain, demonstrating how chatbots and agents can remember previous interactions to maintain context across conversations. Here’s a summary of the key concepts and implementations covered:

---

## 1. **Conversational Memory Overview**
- Conversational memory enables chatbots to reference and utilize previous messages, allowing for more natural and context-aware interactions.
- Without memory, chatbots would only respond to the most recent message, losing all prior context.

---

## 2. **LangChain Memory Types**
We reviewed several memory types available in LangChain (noting that some are being deprecated but are still useful for understanding):

- **ConversationBufferMemory:** Stores the entire conversation history.
- **ConversationBufferWindowMemory:** Stores only the last *k* messages, helping manage token usage and context window limits.
- **ConversationSummaryMemory:** Maintains a summary of the conversation, rather than the full message history.
- **ConversationSummaryBufferMemory:** Combines summary and buffer approaches for efficient memory management.

---

## 3. **Implementing Memory with RunnableWithMessageHistory**
- Introduced the `RunnableWithMessageHistory` class, the recommended approach for managing conversational memory in LangChain.
- Demonstrated how to wrap a pipeline with message history, using session IDs to manage different conversations.
- Showed how to define prompt templates and connect them to LLMs for context-aware responses.

---

## 4. **Storing Memory in External Databases**
- **PostgreSQL (NeonDB):** Demonstrated how to persist chat history in a PostgreSQL database using `PostgresChatMessageHistory`.
- **DynamoDB:** Showed how to create and use a DynamoDB table for storing chat sessions with `DynamoDBChatMessageHistory`.
- These approaches enable scalable, persistent memory across sessions and users.

---

## 5. **Practical Examples**
- Provided code examples for initializing and using each memory type.
- Illustrated how to test memory by invoking pipelines and chains with different session IDs and queries.
- Highlighted the importance of managing token usage and context window size for optimal LLM performance.

---

## 6. **Key Takeaways**
- **Memory is essential** for building effective conversational agents.
- **Different memory types** serve different needs: full history, windowed history, or summarized context.
- **External storage** (like PostgreSQL or DynamoDB) allows for persistent, scalable memory management.
- **RunnableWithMessageHistory** is the modern, flexible way to integrate memory into LangChain workflows.

---